In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
from __future__ import print_function
import seaborn; seaborn.set()
from bot import sendMessage
from copy import copy
import sys

In [99]:
sample = pd.read_csv('submission/ks_r1.csv', index_col='ID')
train = pd.read_csv('data/train.csv', index_col='ID')
test = pd.read_csv('data/test.csv', index_col='ID')

In [73]:
def get_static_categories(train, test, target_col, min_cat_size=1, target_mean=0, cols=None):
    always0 = pd.DataFrame()
    if cols is None:
        cols = train.columns
    for i, col in enumerate(cols):
        if col == target_col:
            continue
        print('\r%.2f'% (100. * i / len(cols)), col, end='')
        sys.stdout.flush()
        cat_mean = train.groupby(col)[target_col].mean() == target_mean
        cat_size = train[col].value_counts()
        if any(cat_mean):
            #always0[col] = test[col].apply(lambda x: cat_mean[x] if x in cat_mean.index and 
            #                               cat_size[x] > min_cat_size else False)
            res0 = np.repeat(False, len(test))
            for val in cat_mean.index:
                if cat_mean[val] and cat_size[val] > min_cat_size:
                    res0[np.where(test[col] == val)] = True  
            always0[col] = res0
        else:
            always0[col] = np.repeat(False, len(test))
    print('\r100.00')
    return always0

a = pd.DataFrame()
a['a'] = [1, 1, 1, 2, 2, 3, 3]
a['t'] = [0, 0, 0, 1, 1, 0, 1]
b = pd.DataFrame()
b['a'] = pd.Series([1, 2, 3, 3, 2, 1])

print(get_static_categories(a, b, 't', 1, 1))


100.00
       a
0  False
1   True
2  False
3  False
4   True
5  False


In [61]:
%%time
always0 = get_static_categories(train, test, 'TARGET')

99.46 var38
Wall time: 14 s


In [79]:
print('target', 1 - train.TARGET.mean())
for mn in range(150, 2000, 1):
    always0 = get_static_categories(train, test, 'TARGET', min_cat_size=mn)
    pc = (always0.sum(axis=1).value_counts() / len(test))[0]
    print(mn, pc)
    if pc > 1 - train.TARGET.mean():
        break
sendMessage('lol')

target 0.960431465404
100.00
150 0.959231317101
100.00
151 0.959231317101
100.00
152 0.96089319159


In [86]:
mn = 152
always0 = get_static_categories(train, test, 'TARGET', min_cat_size=mn)

100.00


In [ ]:
sample.TARGET.iloc[np.where(always0.sum(axis=1) > 0)] = 0
sample.shape

In [90]:
mn = 5
always0_5 = get_static_categories(train, test, 'TARGET', min_cat_size=mn)

100.00


In [100]:
pc = ((always0_5.sum(axis=1) > 4).value_counts() / len(test))
pc

False    0.973964
True     0.026036
dtype: float64

In [101]:
sample.TARGET.iloc[np.where(always0_5.sum(axis=1) > 4)] = 0
sample.shape

(75818, 1)

In [191]:
print(1. * len(int_cat) / len(train.columns))

0.7


In [114]:
int_cat = [cat for cat in train.columns if train[cat].dtype == 'int64']
print('target', 1 - train.TARGET.mean())
for mn in range(0, 160, 10):
    always0 = get_static_categories(train, test, 'TARGET', cols=int_cat, min_cat_size=mn)
    pc = (always0.sum(axis=1).value_counts() / len(test))[0]
    print(mn, pc)
    if pc >= 1:
        break
sendMessage('lol')

target 0.960431465404
100.00
0 0.939921918278
100.00
10 0.949748080931
100.00
20 0.956052652404
100.00
30 0.960708538869
100.00
40 0.965100635733
100.00
50 0.967567068506
100.00
60 0.971049091245
100.00
70 0.971919596929
100.00
80 0.973673797779
100.00
90 0.973673797779
100.00
100 0.973977155821
100.00
110 0.97567859875
100.00
120 0.976008335751
100.00
130 0.976008335751
100.00
140 0.976008335751
100.00
150 0.976008335751


In [174]:
mn = 250
always0 = get_static_categories(train, test, 'TARGET', cols=int_cat, min_cat_size=mn)
#sample = pd.read_csv('submission/ks_r1.csv', index_col='ID')
sample = pd.read_csv('submission/xgb_cl_50bl.csv', index_col='ID')
sample.TARGET.iloc[np.where(always0.sum(axis=1) > 0)] = 0
sample.shape

100.00


(75818, 1)

In [167]:
always0.sum(axis=1).value_counts() / len(test)

0    0.973977
1    0.016197
2    0.007267
4    0.002321
5    0.000119
3    0.000119
dtype: float64

In [ ]:
sendMessage('lol')

In [175]:
sample.to_csv('submission/ks_r1_bt_kor250.csv', index_label='ID')

In [147]:
sample2 = pd.read_csv('submission/ks_r1.csv', index_col='ID') #0.842283
(sample == sample2).sum() / len(sample)
#int mn=295 gives 0.842327

TARGET    0.99574
dtype: float64

In [168]:
sample3 = pd.read_csv('submission/xgb_cl_50bl.csv', index_col='ID')#0.842406
(sample == sample3).sum() / len(sample)
#int 295 gives 0.842430
#int 100 gives 0.840987 (((((

TARGET    0.989567
dtype: float64

In [200]:
uniqs = train.drop(int_cat, axis=1).apply(lambda x: len(x.unique()))
for th in np.linspace(1, 10, 11):
    print('%.0f' % th, '%.2f%%' % (100. * (uniqs > th).sum() / len(uniqs)), len(uniqs) - (uniqs > th).sum())
    print(' '.join([cat for cat in uniqs.index if uniqs[cat] <= th]))

1 100.00% 0

2 100.00% 0

3 98.20% 2
imp_reemb_var17_hace3 saldo_medio_var29_hace3
4 91.89% 9
saldo_var6 saldo_var29 imp_amort_var18_ult1 imp_amort_var34_ult1 imp_reemb_var17_hace3 imp_trasp_var17_in_hace3 saldo_medio_var13_medio_hace2 saldo_medio_var13_medio_ult3 saldo_medio_var29_hace3
5 87.39% 14
saldo_var6 saldo_var29 delta_num_aport_var33_1y3 imp_amort_var18_ult1 imp_amort_var34_ult1 imp_var7_emit_ult1 imp_reemb_var17_hace3 imp_trasp_var17_in_hace3 imp_venta_var44_hace3 saldo_medio_var13_medio_hace2 saldo_medio_var13_medio_ult3 saldo_medio_var29_hace3 saldo_medio_var29_ult1 saldo_medio_var29_ult3
6 82.88% 19
saldo_var6 saldo_var29 delta_imp_venta_var44_1y3 delta_num_aport_var33_1y3 delta_num_venta_var44_1y3 imp_amort_var18_ult1 imp_amort_var34_ult1 imp_var7_emit_ult1 imp_reemb_var17_hace3 imp_trasp_var17_in_hace3 imp_trasp_var17_in_ult1 imp_trasp_var17_out_ult1 imp_venta_var44_hace3 saldo_medio_var13_medio_hace2 saldo_medio_var13_medio_ult3 saldo_medio_var29_hace2 saldo_medio_var2